In [ ]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import datetime as dt

In [ ]:
mp_holistic = mp.solutions.holistic   # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def count_files(folder_path):
    folder_len = len([name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))])
    return folder_len

In [ ]:
def read_file_names(folder_path):
    file_name = [name for name in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, name))]
    return file_name

In [ ]:
def mediapipe_detection(image, model):    
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) # Color conversion from BGR to RGB
    image.flags.writeable = False                  # Image is no longe writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) # Color converson from RGB to BGR
    return image, results

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw Face connections - if we want just outlines of the face
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )

                            # OR if we want mesh
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          )
    
    # Draw Pose connections
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2),
    #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                          )
    
    # Draw Left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(121,44,121), thickness=2, circle_radius=2)
                             )
    # Draw Right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
def extract_keypoints(results):
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [ ]:
# Path for exported data, numpy arrays  
DATA_PATH = os.path.join('MP_Data')   

# Actions that we try to detect  
actions = np.array(['iloveyou', 'no', 'yes'])  

def count_files(folder_path):  
    # Function to count the number of files in a folder  
    return len(os.listdir(folder_path))  

def read_file_names(folder_path):  
    # Function to read file names in a folder  
    return os.listdir(folder_path)  

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:  
    for action in actions:  
        folder_path = "D:\workspace\images\collected_images\handgestures/{}".format(action)  
        file_names = read_file_names(folder_path)  
        
        for sequence in range(len(file_names)):  
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)), exist_ok=True)  
            for frame_num in range(len(file_names)):  
                file_name = file_names[frame_num]  
                file_path = os.path.join(folder_path, file_name)  
                
                image = cv.imread(file_path)  
                if image is None:  
                    print("Error: could not load the image {}".format(file_name))  
                else:  
                    image, results = mediapipe_detection(image, holistic) # Make Detection  
                    draw_styled_landmarks(image, results) # Draw Landmarks  
                    keypoints = extract_keypoints(results) # Extract key points  
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")  
                    np.save(npy_path, keypoints)

In [ ]:
actions = np.array(['hello','thanks','iloveyou', 'no', 'yes'])
DATA_PATH = os.path.join("MP_Data")
for action in actions:  
        folder_path = "D:\workspace\images\collected_images\handgestures/{}".format(action)
sequences, labels = [], []
num_files = count_files(folder_path)
actions = np.array(['hello','thanks','iloveyou', 'no', 'yes'])
label_map = {label:num for num, label in enumerate(actions)}

for action in actions:
    for sequence in range(num_files):
        window = []
        for frame_num in range(num_files):            
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))

            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
sequences

In [ ]:
np.array(sequences[0]).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True, random_state=1)

# LSTM Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(restore_best_weights=True, patience=10)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation="relu", input_shape=(100, 126)))

model.add(LSTM(128, return_sequences=True, activation="relu"))

model.add(LSTM(64, return_sequences=False, activation="relu"))

model.add(Dense(64, activation="relu"))


model.add(Dense(32, activation="relu"))

model.add(Dense(actions.shape[0], activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
log_dir = os.path.join("Logs")
tb_callbacks = TensorBoard(log_dir=log_dir)

In [ ]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callbacks])

In [ ]:
model_eval_loss, model_eval_acc = model.evaluate(X_test, y_test)
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_str = dt.datetime.strftime(current_date_time_dt, date_time_format)
MODEL_PATH = os.path.join("D:\mediapipe_projects\hand_gesture_medium\models")

model_name = f'model__date_time_{current_date_time_str}__loss_{model_eval_loss}__acc_{model_eval_acc}__hand.h5'
model_folder_path = os.path.join(MODEL_PATH, model_name)
model.save(model_folder_path)

In [ ]:
model_name = f'model__date_time_{current_date_time_str}__loss_{model_eval_loss}__acc_{model_eval_acc}__hand.keras'
model_folder_path = os.path.join(MODEL_PATH, model_name)
model.save(model_folder_path)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
import joblib

In [ ]:
date_time_format = '%Y_%m_%d_%H_%M_%S'
current_date_time = dt.datetime.now()
current_date_stamp = dt.datetime.strftime(current_date_time, date_time_format)

In [ ]:
MODEL_PATH = os.path.join("D:\mediapipe_projects\hand_gesture_medium\models")
model_name_pkl = f'model_date_time_{current_date_time_str}__Loss_{model_eval_loss}__acc_{model_eval_acc}_hand.pkl'
model_folder_path = os.path.join(MODEL_PATH, model_name_pkl)
joblib.dump(model, model_folder_path)

In [ ]:
# Python garbage collector to release unused memory by making use of the gc module
import gc
gc.collect()